In [1]:
#imports
import pandas as pd
import scipy 
from scipy import stats
import sklearn.preprocessing
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Collection:

In [ ]:
#imported xlsx files, downloaded from sources

# Preprocessing:

In [ ]:
#PLAN - find yearly avg global temp, add to each entry according to year, find avg estimate for each unique indicator_name in each year, use the appropriate indicator_names to form a score for inequality and train model on x - temp, year | y-inequality scale

df = pd.read_csv('GlobalLandTemperaturesByCountry.csv')
df = df.dropna()
df = df.drop(columns='AverageTemperatureUncertainty')
#dt column -> date: yy/mm/dd
#get an average temp for each year
df['dt'] = pd.to_datetime(df['dt'])
df['year'] = df['dt'].dt.year
yearly_avg_temp = df.groupby('year')['AverageTemperature'].mean().reset_index()

yearly_temps = yearly_avg_temp.set_index('year')['AverageTemperature'].to_numpy()
unique_years_enviro = df['year'].unique()
#to get yearly avg temp for a year -> yearly_temps[desired year - 1743]

df = pd.read_excel('enviro_data.xlsx')
print(df.head())

In [31]:
malaria_df = pd.read_csv('global-malaria.csv')
malaria_df = malaria_df.transpose()
malaria_df.columns = malaria_df.iloc[0]
malaria_df = malaria_df.drop(malaria_df.index[0])
malaria_df.head()

,Year,African Region,Albania,Algeria,American Samoa,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,...,Wales,Western Pacific Region,World,World Bank High Income,World Bank Low Income,World Bank Lower Middle Income,World Bank Upper Middle Income,Yemen,Zambia,Zimbabwe
1990,93,588508,0,9,0,0,5544,0,9,0,...,0,4939,840553,96,369731,455394,15078,5568,8268,3628
1991,189,607330,0,9,0,0,5844,0,9,0,...,0,5247,859283,151,376937,465739,16158,5643,8566,3111
1992,239,620445,0,12,0,0,6183,0,8,0,...,0,5123,856695,174,377923,463295,15027,5693,8724,2210
1993,108,638935,0,12,0,0,6592,0,8,0,...,0,4577,862512,186,385475,463878,12675,5771,8795,2996
1994,211,650973,0,8,0,0,7049,0,8,1,...,0,4660,855983,69,387320,457159,11120,5565,8967,1712


In [ ]:
#target -> health inequality 
#POA -> for each year, find the average estimate for each indicator_name type
# -> then append a yearly_avg_temp column that will contain the avg temp for the year fo the entry
df = df.groupby(['date', 'indicator_name'])['estimate'].mean().reset_index()
print(df.info())
indicator_names = df['indicator_name'].unique()
file = open('unique_values.txt', 'w')
file.write(str(indicator_names))
file.close()

file = open('unique_years.txt', 'w')
file.write(str(unique_years_enviro))
file.close()
#relevant columns from the WHO data -> 'date', 'indicator_name', 'subgroup' 

In [ ]:
df = df[df['date'].isin(unique_years_enviro)]
df = df.iloc[:281] #only have temp up to 2013
avg_temp_year = []
for row in df.itertuples():
    avg_temp_year.append(yearly_temps[row.date - 1784])

df['avg_yearly_temp'] = avg_temp_year
print(df.head())

In [ ]:
#noramlize between 0 and 1
df['estimate'] = (df['estimate'] - df['estimate'].min())/(df['estimate'].max() - df['estimate'].min())

print(df.head())

In [ ]:
yearly_health_ineq_factors = []
estimates = []
start_year = 1990
unique_years_df = [1990]
for row in df.itertuples():
    if row.date == start_year:
        estimates.append(row.estimate)
    else:
        total = sum(estimates)
        total *= (1/len(estimates))
        yearly_health_ineq_factors.append(total)
        estimates = []
        estimates.append(row.estimate)
        start_year += 1
        unique_years_df.append(start_year)
        
estimates = []
for row in df.itertuples():
    if row.date == 2013:
        estimates.append(row.estimate)
        
total = sum(estimates)
total *= (1/len(estimates))
yearly_health_ineq_factors.append(total)

print(unique_years_df)
#put on a scale of 10
stand_factors = [10 * (value - min(yearly_health_ineq_factors)) / (max(yearly_health_ineq_factors) - min(yearly_health_ineq_factors)) for value in yearly_health_ineq_factors]

In [ ]:
#standardize temp
stand_temps = yearly_temps[-24:]
stand_temps = [(value - min(yearly_temps))/(max(yearly_temps) - min(yearly_temps)) for value in stand_temps]

In [ ]:
df = pd.DataFrame({ 'year': list(range(1990, 2014)), 'health_ineq_factor':stand_factors, 'stand_temp':stand_temps, 'temp':yearly_temps[-24:]})
print(df)

In [ ]:
# Summary statistics
df.describe()

In [ ]:
# Convert to numpy arrays for vizualization
stand_factors = np.array(stand_factors)
stand_temps = np.array(stand_temps)
years = np.array(list(range(1990, 2014)))

In [ ]:
# Plot Temperature over time
m, b = np.polyfit(years, stand_temps, 1)

plt.figure(figsize=(10, 6))
plt.plot(years, m * years + b, color='blue', linewidth=2)
plt.scatter(list(range(1990, 2014)), stand_temps, c='red', marker='o', s=50)
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.title('Global Temperature Over Time (Standardized)')
plt.show()

In [ ]:
# Plot inequality over time
plt.scatter(list(range(1990, 2014)), stand_factors, c='red', marker='o', s=50)
plt.xlabel('Time')
plt.ylabel('Health Inequality')
plt.title('Health Inequality Over Time (Standardized)')

plt.show()

In [ ]:
# Plot Inequality dependent on temperature
m, b = np.polyfit(stand_temps, stand_factors, 1)

plt.figure(figsize=(10, 6))
plt.scatter(stand_temps, stand_factors, c='red', marker='o', s=50)
plt.xlabel('Temperature')
plt.ylabel('Health Inequality')
plt.title('Health Inequality and Global Temperature (Standardized)')
plt.show()